In [13]:
import cv2
import numpy as np
from matplotlib import pyplot as plt





#读取要拼接的图片
img1 = cv2.imread('img2.jpg')  
img2 = cv2.imread('img3.jpg')  
imageA = cv2.resize(img1, (0, 0), fx=0.2, fy=0.2)
imageB = cv2.resize(img2, (0, 0), fx=0.2, fy=0.2)

#创建sift对象
sift = cv2.SIFT_create()
#计算关键点，描述符
kp1, descriptors1  = sift.detectAndCompute(imageA, None)
kp2, descriptors2  = sift.detectAndCompute(imageB, None)

#进行两图的特征匹配
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)
good = []

for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append(m)


#计算透视变换矩阵
if len(good) > 20:
    src_points = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    ano_points = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_points, ano_points, cv2.RANSAC, 5.0)
    warpImg = cv2.warpPerspective(imageB, np.linalg.inv(M), (imageA.shape[1] + imageB.shape[1], imageB.shape[0]))
    result = warpImg.copy()
    result[0:imageA.shape[0], 0:imageB.shape[1]] = imageA

rows, cols = imageA.shape[:2]


#找到两张图片的交界线
left_col = np.argwhere(np.any(imageA, axis=0)).min()
right_col = np.argwhere(np.any(imageA, axis=0)).max()

left = left_col.item()
right = right_col.item()

#将图像A和变换后的图像B进行混合
res = np.zeros([rows, cols, 3], np.uint8)
for row in range(0, rows):
    for col in range(0, cols):
        if not imageA[row, col].any():
            res[row, col] = warpImg[row, col]
        elif not warpImg[row, col].any():
            res[row, col] = imageA[row, col]
        else:
            srcImgLen = float(abs(col - left))
            testImgLen = float(abs(col - right))
            alpha = srcImgLen / (srcImgLen + testImgLen)
            res[row, col] = np.clip(imageA[row, col] * (1 - alpha) + warpImg[row, col] * alpha, 0, 255)
warpImg[0:imageA.shape[0], 0:imageA.shape[1]] = res
warpImg[0:imageA.shape[0], 0:imageA.shape[1]] = res


#处理黑边
gray_image = cv2.cvtColor(warpImg, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)
# 使用 findContours 找到非背景区域的轮廓
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# 计算非背景区域的边界矩形
x_min, y_min = np.inf, np.inf
x_max, y_max = -np.inf, -np.inf
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    x_min, y_min = min(x, x_min), min(y, y_min)
    x_max, y_max = max(x + w, x_max), max(y + h, y_max)

# 根据边界矩形裁剪图像，为了去掉黑色背景让图片显示为长方形
cropped_image = warpImg[y_min:y_max, x_min:x_max]



cv2.imshow('Cropped result', cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt


def add_image_to_panorama(panorama, new_image, sift, bf):
    """
    添加新图像到现有全景图。
    Args:
    - panorama: 当前的全景图。
    - new_image: 要添加的新图像。
    - sift: 用于特征检测的SIFT对象。
    - bf: 用于特征匹配的Matcher对象。
    
    Returns:
    - 更新后的全景图。
    """
    # 计算新图像和全景图的关键点和描述符
    kp1, des1 = sift.detectAndCompute(panorama, None)
    kp2, des2 = sift.detectAndCompute(new_image, None)

    # 特征匹配
    matches = bf.knnMatch(des1, des2, k=2)
    good = [m for m, n in matches if m.distance < 0.75 * n.distance]

    # 计算透视变换矩阵
    if len(good) > 10:  # 至少需要10个好的匹配点
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

        # 应用透视变换，创建新全景图
        h, w = panorama.shape[:2]
        h_new, w_new = new_image.shape[:2]
        panorama_warp = cv2.warpPerspective(new_image, M, (w + w_new, h))

        # 更新全景图
        panorama_warp[0:h, 0:w] = panorama
        return panorama_warp
    

def remove_black_background(image):
    # 将图像转换为灰度图
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 创建一个二值图，其中非黑像素被标记为白色（255）
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    # 寻找二值图中的轮廓
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 找到包含所有轮廓的最小矩形
    x_min, y_min, w_min, h_min = cv2.boundingRect(contours[0])
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x < x_min:
            x_min = x
        if y < y_min:
            y_min = y
        if x + w > x_min + w_min:
            w_min = x + w - x_min
        if y + h > y_min + h_min:
            h_min = y + h - y_min

    # 使用计算出的边界矩形裁剪图像
    cropped_image = image[y_min:y_min + h_min, x_min:x_min + w_min]
    return cropped_image

sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

# 假设你已经有一个初始全景图和一些新图像
initial_pano = cv2.imread('img1.jpg')
new_image1 = cv2.imread('img2.jpg')
new_image2 = cv2.imread('img3.jpg')

# 逐步添加图像到全景图
updated_pano = add_image_to_panorama(initial_pano, new_image1, sift, bf)
updated_pano = remove_black_background(updated_pano)
updated_pano = add_image_to_panorama(updated_pano, new_image2, sift, bf)
updated_pano = cv2.resize(updated_pano, (0, 0), fx=0.4, fy=0.4)
updated_pano = remove_black_background(updated_pano)
# 显示结果
cv2.imshow('Updated Panorama', updated_pano)
cv2.waitKey(0)
cv2.destroyAllWindows()